# Getting Data from Iowa state University, UCBerkeley, BYU and Brooklyn YikYak for Analysis
   
**Purpose**
The purpose of this project is to figure out:

* What students like talking about most
* What rate of thier are post postive or negative

**Tools I used**

For this project, I will be using python and R.
__Python__ will be used to access the yikyak data from app and extract the neccessary information we need and save them in a csv file.

__R__ will be used to analyse the text using tidytext and tidyverse libraries.

**Disclaimer:**

The yikyakapi was written by Louis Haddrell which I cloned from gitHub.

Source and Documentation: <a href="https://github.com/louis-haddrell/Yik-Yak-API">click here</a>

In [2]:
import csv
from yikyakapi.yikyak import YikYak
from datetime import datetime

In [6]:
class YiyakData(YikYak):
    
    def __init__(self, PHONE_NUMBER, USER_ID, COUNTRY_CODE = "USA"):
        self.COUNTRY_CODE = COUNTRY_CODE
        self.PHONE_NUMBER = PHONE_NUMBER
        self.USER_ID = USER_ID
        self.client = YikYak()
                
        self.school_cordinates = {"iastate":[42.027288, -93.646422],"ucberkeley":[37.87, -122.2705],
                     "newyork":[40.6782, -73.9442],"byu":[40.251851, -111.649078] }
        try:
            self.client.login_id(self.COUNTRY_CODE, self.PHONE_NUMBER,self.USER_ID)
            print("Singed In")
        except:
            print(str(e))
    
    def _get_school_data(self, school):
        try:
            lat = self.school_cordinates[school][0]
            lng = self.school_cordinates[school][1]
            yaks = self.client.get_new_yaks(lat, lng)
            hot_yaks = self.client.get_hot_yaks(lat, lng)
            
            return ([yak for yak in yaks],[hot for hot in hot_yaks])
        
        except Exception as e:
            return {"error":str(e)}
        
    def get_current(self,school):
        yak, hot = self._get_school_data(school)
        return {"current_yaks":str(yak)[0:4], "hot":str(hot)[0:4]}
    
    def get_length(self,school):
        yak_length, hot_length = self._get_school_data(school)
        return {"current":len(yak_length), "hot":len(hot_length)}

    def save_yak(self,school):
        yaks, hot_yaks = self._get_school_data(school)
        
        #extract the data from yikyak
        yak_results = {str(message):yaks[indx].comments for indx, message in enumerate(yaks)}
        hot_results = {str(message):hot_yaks[indx].comments for indx, message in enumerate(hot_yaks)}
        #time_posted = {indx:yaks[indx].time for indx, dt in enumerate(yaks)}
        
        #save our data into a csv file
        file_ext=str(datetime.strftime(datetime.now(),"%Y_%m_%d_%H%M%S")) + ".csv"
        file_and_data = [(school+file_ext ,yak_results), ('hot'+school+file_ext,hot_results)] #message_comments
        

        #write the messages and the number of comments
        for (file_name, data) in file_and_data:
            #file_name = str(file_name + "_" + str(school) + "_" + file_ext)

            with open(file_name , "w") as f:
                fieldnames = ['Message', "Number_of_replies"]
                writer = csv.DictWriter(f, fieldnames=fieldnames)

                writer.writeheader()
                for k, v in data.items():
                    writer.writerow({"Message":k, "Number_of_replies":v})

        return {"results":[{"school_name":str(school)},
                           {"status":"data saved"},
                           {"num_of_yak":len(yak_results)},
                           {"num_of_hot_yaks":len(hot_results)}
                          ]}
       

In [7]:
data = YiyakData(PHONE_NUMBER = "", USER_ID = "") # you would have to provide your credentials
print(data.save_yak("iastate"))
print(data.save_yak("ucberkeley"))
print(data.save_yak("byu"))
print(data.save_yak("newyork"))

Singed In
{'results': [{'school_name': 'iastate'}, {'status': 'data saved'}, {'num_of_yak': 92}, {'num_of_hot_yaks': 100}]}
{'results': [{'school_name': 'ucberkeley'}, {'status': 'data saved'}, {'num_of_yak': 69}, {'num_of_hot_yaks': 98}]}
{'results': [{'school_name': 'byu'}, {'status': 'data saved'}, {'num_of_yak': 100}, {'num_of_hot_yaks': 100}]}
{'results': [{'school_name': 'newyork'}, {'status': 'data saved'}, {'num_of_yak': 100}, {'num_of_hot_yaks': 99}]}
